<a href="https://colab.research.google.com/github/beyzaturku/image_processing_in_autonomous_vehicles/blob/main/lane_detectin_and_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q opencv-python

In [ ]:
# Libraries for working with image processing
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
# Libraries needed to edit/save/watch video clips
from moviepy import editor
import moviepy

  if event.key is 'enter':



In [ ]:
test_video = '/content/project_video.mp4'
output_video = '/content/output_video.mp4'

In [ ]:
import cv2
import numpy as np
import moviepy.editor as editor
import time
import os
from scipy.interpolate import UnivariateSpline

# 0. Işık koşuluna göre analiz
def detect_light_condition(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean = np.mean(gray)
    if mean < 60:
        return 'dark'
    elif mean > 180:
        return 'bright'
    else:
        return 'normal'

# CLAHE histogram eşitleme
def equalize_gray(gray):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(gray)

# Dinamik lane analiz fonksiyonu
def analyze_lanes(img, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    widths = []
    intensities = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h > 30 and w < 50:
            widths.append(w)
            region = img[y:y+h, x:x+w]
            intensities.append(np.mean(region))
    avg_width = np.mean(widths) if widths else None
    avg_intensity = np.mean(intensities) if intensities else None
    return avg_width, avg_intensity

# 1. Adaptif renk eşikleme + dinamik analiz
def color_threshold(img):
    condition = detect_light_condition(img)
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    temp_mask = cv2.inRange(hls[:, :, 1], 180, 255)
    width, intensity = analyze_lanes(img, temp_mask)

    if intensity and intensity < 100:
        white = cv2.inRange(hls[:, :, 1], 150, 230)
    else:
        white = cv2.inRange(hls[:, :, 1], 200, 255)

    if condition == 'dark':
        yellow = cv2.inRange(hls[:, :, 0], 10, 40) & cv2.inRange(hls[:, :, 2], 90, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 30), (180, 80, 100))
    elif condition == 'bright':
        yellow = cv2.inRange(hls[:, :, 0], 20, 35) & cv2.inRange(hls[:, :, 2], 120, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 70), (180, 90, 200))
    else:
        yellow = cv2.inRange(hls[:, :, 0], 15, 35) & cv2.inRange(hls[:, :, 2], 100, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 40), (180, 70, 150))

    light = cv2.inRange(lab[:, :, 0], 180, 255)
    dark_road = cv2.inRange(hsv, (0, 0, 40), (180, 60, 120))

    bright_spots = cv2.inRange(lab[:, :, 0], 240, 255)

    mask = cv2.bitwise_or(white, yellow)
    mask = cv2.bitwise_or(mask, light)
    mask = cv2.bitwise_or(mask, adaptive_road)
    mask = cv2.bitwise_or(mask, dark_road)
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(bright_spots))

    globals()['CURRENT_MIN_LINE_LENGTH'] = 10 if width and width < 5 else 30
    return mask

# 2. Kenar tespiti
def edge_detection(gray):
    equalized = equalize_gray(gray)
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

# 3. ROI
def region_of_interest(img):
    h, w = img.shape[:2]
    mask = np.zeros_like(img)
    polygon = np.array([[
        (int(w * 0.1), int(h * 0.95)),
        (int(w * 0.4), int(h * 0.65)),
        (int(w * 0.6), int(h * 0.65)),
        (int(w * 0.9), int(h * 0.95))
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

# 4. Hough
def hough_lines(img):
    min_length = globals().get('CURRENT_MIN_LINE_LENGTH', 20)
    lines = cv2.HoughLinesP(img, 1, np.pi / 180, 30, minLineLength=min_length, maxLineGap=120)
    if lines is None:
        return []
    filtered = []
    for x1, y1, x2, y2 in lines.reshape(-1, 4):
        if x2 == x1:
            continue
        slope = (y2 - y1) / (x2 - x1)
        angle = abs(np.degrees(np.arctan(slope)))
        if 10 < angle < 85:
            filtered.append((x1, y1, x2, y2, slope))
    return filtered

# 5. Regresyon + spline ile eğri çizgi çizimi
def fit_lane_lines(img, lines):
    h, w = img.shape[:2]
    left_points, right_points = [], []

    for x1, y1, x2, y2, slope in lines:
        if slope < 0:
            left_points += [(x1, y1), (x2, y2)]
        else:
            right_points += [(x1, y1), (x2, y2)]

    lane_img = np.zeros_like(img)
    car_offset = None

    def draw_spline(points, color):
        if len(points) < 4:
            return None
        x, y = zip(*points)
        y = np.array(y)
        x = np.array(x)
        spline = UnivariateSpline(y, x, k=2, s=5)
        y_range = np.linspace(min(y), max(y), num=50)
        x_smooth = spline(y_range).astype(np.int32)
        pts = np.array([[xv, int(yv)] for xv, yv in zip(x_smooth, y_range)], dtype=np.int32)
        for i in range(len(pts) - 1):
            cv2.line(lane_img, tuple(pts[i]), tuple(pts[i+1]), color, 8)
        return pts[0][0]  # return base x

    left_base = draw_spline(left_points, (0, 255, 0))
    right_base = draw_spline(right_points, (255, 0, 0))

    if left_base is not None and right_base is not None:
        lane_center = (left_base + right_base) // 2
        car_center = w // 2
        car_offset = (car_center - lane_center) * 3.7 / 700

    return lane_img, car_offset


# FPS ölçümü
last_time = [time.time()]
frame_counter = [0]

def frame_processor(image):
    start_time = time.time()
    frame_id = frame_counter[0]
    frame_counter[0] += 1
    os.makedirs("output", exist_ok=True)

    color_mask = color_threshold(image)
    cv2.imwrite(f"output/{frame_id:03d}_1_color_mask.jpg", color_mask)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edge_mask = edge_detection(gray)
    cv2.imwrite(f"output/{frame_id:03d}_2_edges.jpg", edge_mask)

    combined = cv2.bitwise_or(color_mask, edge_mask)
    cv2.imwrite(f"output/{frame_id:03d}_3_combined.jpg", combined)

    roi = region_of_interest(combined)
    cv2.imwrite(f"output/{frame_id:03d}_4_roi.jpg", roi)

    lines = hough_lines(roi)
    hough_img = np.zeros_like(image)
    for x1, y1, x2, y2, _ in lines:
        cv2.line(hough_img, (x1, y1), (x2, y2), (0, 255, 255), 2)
    cv2.imwrite(f"output/{frame_id:03d}_5_hough_lines.jpg", hough_img)

    lane_img, offset = fit_lane_lines(image, lines)
    overlay = cv2.addWeighted(image, 0.8, lane_img, 1, 0)

    fps = 1.0 / (start_time - last_time[0])
    last_time[0] = start_time

    cv2.putText(overlay, f"FPS: {fps:.1f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    if offset is not None:
        direction = "left" if offset > 0 else "right"
        cv2.putText(overlay, f"Offset: {abs(offset):.2f}m {direction}", (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    cv2.imwrite(f"output/{frame_id:03d}_6_final_overlay.jpg", overlay)
    return overlay

def process_video(input_path, output_path):
    clip = editor.VideoFileClip(input_path)
    result = clip.fl_image(frame_processor)
    result.write_videofile(output_path, audio=False)






In [ ]:
# Çalıştırma
process_video(test_video, output_video)


ValueError: x must be increasing if s > 0

In [ ]:
import cv2
import numpy as np
import moviepy.editor as editor
import time
import os
from scipy.interpolate import UnivariateSpline

# 0. Işık koşuluna göre analiz
def detect_light_condition(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean = np.mean(gray)
    if mean < 60:
        return 'dark'
    elif mean > 180:
        return 'bright'
    else:
        return 'normal'

# CLAHE histogram eşitleme
def equalize_gray(gray):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(gray)

# Dinamik lane analiz fonksiyonu
def analyze_lanes(img, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    widths = []
    intensities = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if h > 30 and w < 50:
            widths.append(w)
            region = img[y:y+h, x:x+w]
            intensities.append(np.mean(region))
    avg_width = np.mean(widths) if widths else None
    avg_intensity = np.mean(intensities) if intensities else None
    return avg_width, avg_intensity

# 1. Adaptif renk eşikleme + dinamik analiz
def color_threshold(img):
    condition = detect_light_condition(img)
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    temp_mask = cv2.inRange(hls[:, :, 1], 180, 255)
    width, intensity = analyze_lanes(img, temp_mask)

    if intensity and intensity < 100:
        white = cv2.inRange(hls[:, :, 1], 150, 230)
    else:
        white = cv2.inRange(hls[:, :, 1], 200, 255)

    if condition == 'dark':
        yellow = cv2.inRange(hls[:, :, 0], 10, 40) & cv2.inRange(hls[:, :, 2], 90, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 30), (180, 80, 100))
    elif condition == 'bright':
        yellow = cv2.inRange(hls[:, :, 0], 20, 35) & cv2.inRange(hls[:, :, 2], 120, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 70), (180, 90, 200))
    else:
        yellow = cv2.inRange(hls[:, :, 0], 15, 35) & cv2.inRange(hls[:, :, 2], 100, 255)
        adaptive_road = cv2.inRange(hsv, (0, 0, 40), (180, 70, 150))

    light = cv2.inRange(lab[:, :, 0], 180, 255)
    dark_road = cv2.inRange(hsv, (0, 0, 40), (180, 60, 120))

    bright_spots = cv2.inRange(lab[:, :, 0], 240, 255)

    mask = cv2.bitwise_or(white, yellow)
    mask = cv2.bitwise_or(mask, light)
    mask = cv2.bitwise_or(mask, adaptive_road)
    mask = cv2.bitwise_or(mask, dark_road)
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(bright_spots))

    globals()['CURRENT_MIN_LINE_LENGTH'] = 10 if width and width < 5 else 30
    return mask

# 2. Kenar tespiti
def edge_detection(gray):
    equalized = equalize_gray(gray)
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

# 3. ROI
def region_of_interest(img):
    h, w = img.shape[:2]
    mask = np.zeros_like(img)
    polygon = np.array([[
        (int(w * 0.1), int(h * 0.95)),
        (int(w * 0.4), int(h * 0.65)),
        (int(w * 0.6), int(h * 0.65)),
        (int(w * 0.9), int(h * 0.95))
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

# 4. Hough
def hough_lines(img):
    min_length = globals().get('CURRENT_MIN_LINE_LENGTH', 20)
    lines = cv2.HoughLinesP(img, 1, np.pi / 180, 30, minLineLength=min_length, maxLineGap=120)
    if lines is None:
        return []
    filtered = []
    for x1, y1, x2, y2 in lines.reshape(-1, 4):
        if x2 == x1:
            continue
        slope = (y2 - y1) / (x2 - x1)
        angle = abs(np.degrees(np.arctan(slope)))
        if 10 < angle < 85:
            filtered.append((x1, y1, x2, y2, slope))
    return filtered

# 5. Regresyon + spline ile eğri çizgi çizimi
def fit_lane_lines(img, lines):
    h, w = img.shape[:2]
    left_points, right_points = [], []

    for x1, y1, x2, y2, slope in lines:
        if slope < 0:
            left_points += [(x1, y1), (x2, y2)]
        else:
            right_points += [(x1, y1), (x2, y2)]

    lane_img = np.zeros_like(img)
    car_offset = None

    def draw_spline(points, color):
        if len(points) < 4:
            return None
        x, y = zip(*points)
        y = np.array(y)
        x = np.array(x)
        spline = UnivariateSpline(y, x, k=2, s=5)
        y_range = np.linspace(min(y), max(y), num=50)
        x_smooth = spline(y_range).astype(np.int32)
        pts = np.array([[xv, int(yv)] for xv, yv in zip(x_smooth, y_range)], dtype=np.int32)
        for i in range(len(pts) - 1):
            cv2.line(lane_img, tuple(pts[i]), tuple(pts[i+1]), color, 8)
        return pts[0][0]  # return base x

    left_base = draw_spline(left_points, (0, 255, 0))
    right_base = draw_spline(right_points, (255, 0, 0))

    if left_base is not None and right_base is not None:
        lane_center = (left_base + right_base) // 2
        car_center = w // 2
        car_offset = (car_center - lane_center) * 3.7 / 700

    return lane_img, car_offset
